In [5]:
import os

In [6]:
%pwd

'm:\\Learning_Workspace\\AI\\AI_Projects\\KrishNaikProjects\\End-To-End-ML-Project-MLflow-AWS\\research'

In [7]:
os.chdir('../')

In [8]:
%pwd

'm:\\Learning_Workspace\\AI\\AI_Projects\\KrishNaikProjects\\End-To-End-ML-Project-MLflow-AWS'

In [1]:
import dagshub
dagshub.init(repo_owner='marthalarajavardhanreddy', repo_name='End-To-End-ML-Project-MLflow-AWS', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=d9c5aa18-d22a-48b6-a900-8fc9967d8d17&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=775a3939aa6937ff26abe085f23ec1c9c5c3b364784c820b48512cbc965d4224




Accessing as marthalarajavardhanreddy

Initialized MLflow to track repo "marthalarajavardhanreddy/End-To-End-ML-Project-MLflow-AWS"

Repository marthalarajavardhanreddy/End-To-End-ML-Project-MLflow-AWS initialized!

In [62]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/marthalarajavardhanreddy/End-To-End-ML-Project-MLflow-AWS.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="marthalarajavardhanreddy"
os.environ["MLFLOW_TRACKING_PASSWORD"]="9232db7eb3497e8eb8bee9b441a9eccec6c77dec"

In [63]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [64]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [65]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/marthalarajavardhanreddy/End-To-End-ML-Project-MLflow-AWS.mlflow",
        )
        return model_evaluation_config


In [66]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from src.mlProject.utils.common import save_json


In [67]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [68]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-07-18 13:42:22,492: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-18 13:42:22,496: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-18 13:42:22,502: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-18 13:42:22,506: INFO: common: created directory: artifacts]
[2024-07-18 13:42:22,508: INFO: common: created directory: artifacts/model_evaluation]
[2024-07-18 13:42:23,663: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


m:\Fliq_Workspace\installations\AI\envs\space-AI\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
m:\Fliq_Workspace\installations\AI\envs\space-AI\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2024/07/18 13:42:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 7
Created version '7'